# Trying to train UNet with L1 loss

In [ ]:
!pip install scikit-image
!pip install IProgress
!pip install jupyter
!pip install ipywidgets widgetsnbextension pandas-profiling
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
dataset = "/datasets/coco"
version = "2014"

In [ ]:
import torch
import torch.utils.data as data
from python.data.dataset import *
from python.models.generator import *
from python.utils.images import *
from python.train.trainer import *
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
dataset_train = CocoLab(dataset, version=version, size=256, train=True)
trainloader = data.DataLoader(dataset_train, batch_size=4, shuffle=True, num_workers=2)

dataset_test = CocoLab(dataset, version=version, size=256, train=False)
testloader = data.DataLoader(dataset_test, batch_size=4, shuffle=True, num_workers=2)

generator = UNet(1, 2).to(device)

In [ ]:
# one image from the training set
L_base, ab_base = next(iter(trainloader))

Lab = torch.concat((L_base, ab_base), 1)
tensor_to_pil(Lab)[0]

In [ ]:
# trying the prediction of the Unet before training
generator.eval()
L_base = L_base.to(device)
ab_pred_notrain = generator(L_base).detach()
Lab_pred_notrain = torch.concat((L_base, ab_pred_notrain), 1).to("cpu")
tensor_to_pil(Lab_pred_notrain)[0]

In [ ]:
num_epochs = 10

In [ ]:
LEARNING_RATE = 0.0002

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(generator.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

train_avg_loss = []
test_avg_loss = []

for epoch in range(num_epochs):
    train_losses = []
    test_losses = []
    
    generator.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for L, ab in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
            L = L.to(device)
            ab = ab.to(device)

            pred = generator(L)
            loss = criterion(pred, ab)

            train_losses.append(loss.detach())
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

        with torch.no_grad():   
            generator.eval()

            for L, ab in testloader:
                L = L.to(device)
                ab = ab.to(device)

                pred = generator(L)
                loss = criterion(pred, ab)
                test_losses.append(loss.detach())

            train_avg_loss.append(torch.mean(torch.Tensor(train_losses)))
            test_avg_loss.append(torch.mean(torch.Tensor(test_losses)))

            print('[Epoch {}/{}] '.format(epoch+1, num_epochs) +
                    'train_loss: {:.4f} - '.format(train_avg_loss[-1]) +
                    'test_loss: {:.4f}'.format(test_avg_loss[-1]))


            generator.eval()
            ab_pred = generator(L_base).detach()
            Lab_pred = torch.concat((L_base, ab_pred), 1).to("cpu")
            tensor_to_pil(Lab_pred)[0].save("epoch_{}.png".format(epoch))
            torch.save(generator.state_dict(), "saved_models/generator")


In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.title('Losses')
plt.plot(train_avg_loss)
plt.plot(test_avg_loss)
plt.grid()
plt.legend(['Train', 'Test'])
plt.xlabel('Epoch')
plt.ylabel('Loss (L1)')


plt.show()

In [ ]:
# trying the prediction of the Unet after training
generator.eval()
ab_pred_train = generator(L).detach()
Lab_pred_train = torch.concat((L, ab_pred_train), 1).to("cpu")

In [ ]:
tensor_to_pil(Lab_pred_train)[0]